In [ ]:
# Requirements
pip install kserve

In [ ]:
from kserve import RESTConfig, InferenceRESTClient, InferRequest, InferInput
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from random import randrange
import numpy as np
import os
import json
import requests
import io

In [ ]:
# Download dataset
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip

In [ ]:
%%sh
unzip -q dogImages.zip
mv dogImages dog-breeds-dataset

In [ ]:
# Dataset
ds_dog_breeds_path = "./dog-breeds-dataset"
test_ds_dog_breeds_path = os.path.join(ds_dog_breeds_path, "test")
ds_dog_breeds = datasets.ImageFolder(test_ds_dog_breeds_path)
dog_breeds_classes = ds_dog_breeds.find_classes(test_ds_dog_breeds_path)[0]

print(f"Classes dog_breeds_classifier: {dog_breeds_classes}")

In [ ]:
# Show sample image
fig = plt.figure(figsize=(1, 1))
sample_idx = randrange(len(ds_dog_breeds))

img, label = ds_dog_breeds[sample_idx]
fig.add_subplot(1, 1, 1)

plt.title(dog_breeds_classes[label])
plt.axis("off")
plt.imshow(img)
plt.show()

In [ ]:
img_array = np.array(img.convert('RGB'))

if img_array.ndim == 3:
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
print(img_array.shape)


In [ ]:
# Inference
model_name = "dog-breeds-classifier"
request = InferRequest(
    model_name=model_name,
    infer_inputs=[
        InferInput(
            name="data",
            shape=list(img_array.shape),
            datatype="FP32",
            data=img_array.tolist()
        )
    ]
)

cfg = RESTConfig(protocol="v2", verify=False)
client = InferenceRESTClient(cfg)

base_url = "https://dog-breeds-classifier-modelserving-demo.apps.rosa.n1t3u2f3w1s0b1d.kkw2.p3.openshiftapps.com"

In [ ]:
# Note: May throw an error when using the inference graph pipeline if the input is an image of a cat
# This is because the return code is 500
infer_response = await client.infer(base_url, request, request.model_name, is_graph_endpoint=False)
det_class = dog_breeds_classes[np.argmax(infer_response.outputs[0].data)]
print("Raw response: ", infer_response)
print(f"Detected class: {det_class}")